# 02 VAR and Impulse Responses

Fit VARs, test Granger causality, and interpret IRFs.


## Table of Contents
- [Build stationary dataset](#build-stationary-dataset)
- [Fit VAR + choose lags](#fit-var-choose-lags)
- [Granger causality](#granger-causality)
- [IRFs + forecasting](#irfs-forecasting)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Time-series econometrics notebooks build the classical toolkit you need before trusting macro regressions:
- stationarity + unit roots,
- cointegration + error correction,
- VAR dynamics and impulse responses.


## Prerequisites (Quick Self-Check)
- Completed Part 01 macro panel notebooks (or have `panel_monthly.csv` / sample available).
- Comfort with differencing/log transforms and reading time series plots.

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Running tests without plotting or transforming the series first.
- Treating impulse responses as structural causality without an identification story.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/07_time_series_econ/02_var_impulse_responses.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/07_time_series_econ/02_var_impulse_responses.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Fit a VAR on transformed macro series and interpret:
- lag selection
- Granger causality
- impulse response functions (IRFs)



## Primer: Classical time-series econometrics with `statsmodels` (ADF/KPSS, cointegration, VAR/IRF)

This repo already teaches time-aware evaluation for ML. This primer introduces the classical econometrics toolkit for time series:
- stationarity / unit roots,
- cointegration and error correction ideas,
- VARs and impulse responses.

Deep theory is in the guides; this primer focuses on “how to use the tools correctly.”

### Before you start: what you should always do

1) **Plot the series in levels** (look for trends, breaks).
2) **Choose transformations** (diff/logdiff) for stationarity.
3) **Drop missing values** before tests/models.

### Stationarity tests (ADF / KPSS)

Two common tests:
- **ADF**: null = unit root (nonstationary)
- **KPSS**: null = stationary

```python
from statsmodels.tsa.stattools import adfuller, kpss

x = df["SERIES"].dropna().to_numpy()

adf_stat, adf_p, *_ = adfuller(x)
kpss_stat, kpss_p, *_ = kpss(x, regression="c", nlags="auto")

print("ADF p:", adf_p, "KPSS p:", kpss_p)
```

**Expected output / sanity check**
- trending level series often: ADF p not small, KPSS p small
- differenced series often: ADF p small, KPSS p not small

### Cointegration (Engle–Granger test)

If two series are individually nonstationary but move together long-run, they may be cointegrated.

```python
from statsmodels.tsa.stattools import coint

y = df["Y"].dropna()
x = df["X"].dropna()

stat, p, _ = coint(y, x)
print("coint p:", p)
```

### VAR (vector autoregression)

VAR models multiple stationary-ish series jointly.

```python
from statsmodels.tsa.api import VAR

X = df[["UNRATE", "FEDFUNDS", "INDPRO"]].astype(float).dropna()
X = X.diff().dropna()  # common stationarity transform

model = VAR(X)
res = model.fit(maxlags=8, ic="aic")  # or choose lags manually
print("lags chosen:", res.k_ar)
print(res.summary())
```

**Expected output / sanity check**
- `res.k_ar` is the chosen lag length
- `res.is_stable(verbose=False)` should be True for a stable VAR

### Granger causality (predictive, not causal)

```python
res.test_causality("UNRATE", ["FEDFUNDS"]).summary()
```

Interpretation: “do lagged FEDFUNDS help predict UNRATE beyond lagged UNRATE?”

### Impulse responses (IRFs)

```python
irf = res.irf(12)
irf.plot(orth=True)  # orthogonalized IRFs (ordering matters)
```

**Important:** orthogonalized IRFs depend on a Cholesky ordering.

### Common pitfalls (and quick fixes)

- **Nonstationary inputs:** VAR on levels can be nonsense.
  - Fix: difference/logdiff; or use cointegration/VECM logic.
- **Too many lags:** eats degrees of freedom and can destabilize the model.
  - Fix: try smaller maxlags, compare AIC/BIC, check diagnostics.
- **Misinterpreting Granger causality:** it is about predictive content, not structural causality.
- **Forgetting ordering:** orth IRFs change when you reorder variables.


<a id="build-stationary-dataset"></a>
## Build stationary dataset

### Background
VARs generally require stable (stationary-ish) inputs.
A common first pass is to difference level series.

### What you should see
- a DataFrame of transformed series (no missing values).
- columns are numeric floats.

### Interpretation prompts
- What does differencing do to trends and to noise?
- Which series might require log-differencing rather than differencing?

### Goal
Build a small stationary-ish dataset to fit a VAR.



### Your Turn: Load and transform


In [ ]:
import pandas as pd

panel = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True).dropna()

# TODO: Choose a few columns and difference them
df = panel[['UNRATE', 'FEDFUNDS', 'INDPRO']].astype(float).diff().dropna()
df.head()



<a id="fit-var-choose-lags"></a>
## Fit VAR + choose lags

### Background
VAR lag length is a bias–variance decision:
- too few lags → leftover autocorrelation and misspecification,
- too many lags → unstable estimates and low degrees of freedom.

### What you should see
- a chosen lag order (`res.k_ar`).
- a model summary with coefficients for lagged terms.

### Interpretation prompts
- Why might AIC choose more lags than BIC?
- What diagnostic would you check if you suspect too few lags?

### Goal
Fit a VAR and choose lags using an information criterion.



### Your Turn: Fit VAR


In [ ]:
from statsmodels.tsa.api import VAR

# TODO: Fit and inspect chosen lag order
res = VAR(df).fit(maxlags=8, ic='aic')
res.k_ar



<a id="granger-causality"></a>
## Granger causality

### Background
Granger causality asks a forecasting question:
- do lagged values of $x$ help predict $y$ beyond lagged $y$?

It is not structural causality.

### What you should see
- a test output summary for one direction (e.g., FEDFUNDS → UNRATE).

### Interpretation prompts
- Rewrite the Granger test result as a forecasting statement (not a causal one).
- Why can a third variable create apparent Granger relationships?

### Goal
Run at least one Granger causality test.

Reminder: this is predictive causality, not structural causality.



### Your Turn: Test causality


In [ ]:
# Example: do lagged FEDFUNDS help predict UNRATE?
res.test_causality('UNRATE', ['FEDFUNDS']).summary()



## IRFs + forecasting

### Background
Impulse responses trace how a one-time shock propagates through the VAR dynamics.
Orthogonalized IRFs (Cholesky) impose an identification choice via variable ordering.

### Reduced-form vs Structural VARs

The VAR estimated here is a **reduced-form** VAR: it captures correlations and dynamics but does not identify structural shocks. The Cholesky decomposition imposes a recursive ordering assumption:
- The first variable responds to its own shock contemporaneously.
- The second variable can respond to the first variable's shock contemporaneously, but the first cannot respond to the second's.
- And so on.

A **Structural VAR (SVAR)** uses economic theory to justify identification restrictions (e.g., short-run zero restrictions, long-run restrictions, or sign restrictions). SVARs are beyond this notebook's scope, but you should know they exist when interpreting IRFs.

### Granger causality: what it is and what it is NOT

Granger causality is a statistical concept: "$x$ Granger-causes $y$" means lagged $x$ has predictive content for $y$ beyond lagged $y$ itself. This is **not structural causality**:
- A third variable $z$ that drives both $x$ and $y$ can create apparent Granger causality.
- The Fed may set rates in response to expected unemployment, creating a statistical relationship that reverses the causal direction.
- Granger causality depends on the information set (which variables are in the VAR).

**Rule**: Always write Granger causality results as forecasting statements, never as causal claims.

### What you should see
- an IRF plot over the chosen horizon.
- qualitative responses that decay if the VAR is stable.

### Interpretation prompts
- How does changing the variable ordering change the meaning of the shock?
- Which IRF responses would you view as economically plausible vs suspicious?

### Goal
Compute and plot impulse responses.

Caution:
- orthogonalized IRFs depend on variable ordering.


### Your Turn: IRFs


In [ ]:
irf = res.irf(12)
irf.plot(orth=True)



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# TODO: Validate your time series table is well-formed.
# Example (adjust variable names):
# assert isinstance(df.index, pd.DatetimeIndex)
# assert df.index.is_monotonic_increasing
# assert df.shape[0] > 30
#
# TODO: If you built transformed series (diff/logdiff), confirm no future leakage.
# Hint: transformations should only use past/current values (shift/diff), never future.
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Build stationary dataset</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_var_impulse_responses — Build stationary dataset
import pandas as pd

panel = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True).dropna()
df = panel[['UNRATE', 'FEDFUNDS', 'INDPRO']].astype(float).diff().dropna()
df.head()
```

</details>

<details><summary>Solution: Fit VAR + choose lags</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_var_impulse_responses — Fit VAR + choose lags
from statsmodels.tsa.api import VAR

res = VAR(df).fit(maxlags=8, ic='aic')
res.k_ar
```

</details>

<details><summary>Solution: Granger causality</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_var_impulse_responses — Granger causality
# Example: do lagged FEDFUNDS help predict UNRATE?
res.test_causality('UNRATE', ['FEDFUNDS']).summary()
```

</details>

<details><summary>Solution: IRFs + forecasting</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_var_impulse_responses — IRFs + forecasting
irf = res.irf(12)
irf.plot(orth=True)
```

</details>

